In [1]:
import json
from textblob import TextBlob

class Tweet:
    """
    Cette classe permet de créer un objet tweet.
    """
    
    cleaned_text: list # Liste des mots du tweet
    
    def __init__(self, tweet_data: dict):
        """
        Constructeur de la classe Tweet
        :param tweet_data: Dictionnaire contenant les données du tweet
        """
        
        self.id = tweet_data["id"]
        self.text = tweet_data["TweetText"]
        self.clean_text(["#", "@"])
        self.hashtags = []
        self.arobase = []
        self.feelings = self.calculate_sentiment()


    def calculate_sentiment(self) -> str:
        """
        Cette fonction permet de calculer le sentiment du tweet.
        :return bool: True si sentiment positif, False sinon
        """
        return "Négatif" if TextBlob(self.text).sentiment.polarity < 0 else "Positif"

    def extract_entities(self):
        """
        Cette fonction permet d'extraire les entités du tweet (hashtags, arobase)
        """
        
        # On parcourt chaque mot du tweet
        for word in self.cleaned_text:
            if word[0] == "#": # Si le mot commence par un #
                self.hashtags.append(word[1:]) # On ajoute le mot sans le # à la liste des hashtags
            elif word[0] == "@": # Si le mot commence par un @
                self.arobase.append(word[1:]) # On ajoute le mot sans le @ à la liste de l'arobase

    def clean_text(self, excepted_char: list):
        """
        Cette fonction permet de nettoyer le texte du tweet.
        :param excepted_char: Liste des caractères à ne pas remplacer
        """
        
        # On remplace les caractères spéciaux par des espaces sauf si ceux-ci sont dans excepted_char
        cleaned_text = self.text.translate({
            char: " " for char in self.text if not char.isalnum() and char not in excepted_char
        })
    
        # On remplace ":" par un espace
        cleaned_text = cleaned_text.replace(":", " ")
        cleaned_text = cleaned_text.replace("#…", " ")
        cleaned_text = cleaned_text.replace("…", " ")
        cleaned_text = cleaned_text.replace(".", " ")
        cleaned_text = cleaned_text.replace("!", " ")
    
        # On sépare le texte en liste de mots
        self.cleaned_text = [word for word in cleaned_text.strip().split()]

In [2]:
# Cellule qui gère les fonctions sur les hashtags

def count_hashtags(tweets: list) -> dict:
    """
    Cette fonction permet de retourner les hashtags avec leur nombre de publications.
    :param tweets: Liste des tweets
    :return hashtags: Dictionnaire des hashtags avec leur nombre de publications
    """
    
    hashtags = {} # Dictionnaire des hashtags
    
    # On parcourt chaque tweet
    for tweet in tweets:
        # On parcourt chaque hashtag du tweet
        for hashtag in tweet.hashtags:
            if hashtag in hashtags: # Si le hashtag est déjà dans le dictionnaire
                hashtags[hashtag] += 1 # On incrémente le nombre d'occurence du hashtag
            else: # Sinon
                hashtags[hashtag] = 1 # On ajoute le hashtag au dictionnaire
        
    # On retourne les hashtags avec leur nombre de publications
    return hashtags

def top_hashtags(k:int, hashtags: dict) -> list:
    """
    Cette fonction permet de retourner les k hashtags les plus utilisés.
    :param k: Nombre de hashtags à retourner
    :param hashtags: Liste des hashtags avec leur nombre de publications
    :return top_hashtags: Liste des k hashtags les plus utilisés
    """
    
    # On retourne les k hashtags les plus utilisés
    return sorted(hashtags, key=hashtags.get, reverse=True)[:k]

In [3]:
# Cellule qui gère les fonctions sur les arobases

def count_arobases(tweets: list) -> dict:
    """
    Cette fonction permet de retourner les arobases avec leur nombre de publications.
    :param tweets: Liste des tweets
    :return arobases: Dictionnaire des k arobase les plus utilisés
    """
    
    arobases = {} # Dictionnaire de l'arobase
    
    # On parcourt chaque tweet
    for tweet in tweets:
        # On parcourt chaque arobase du tweet
        for arobase in tweet.arobase:
            if arobase in arobases: # Si l'arobase est déjà dans le dictionnaire
                arobases[arobase]["occurence"] += 1 # On incrémente le nombre d'occurence de l'arobase
                arobases[arobase]["id"].append(tweet.id) # On ajoute l'id du tweet à la liste des id de l'arobase
            else: # Sinon
                # On ajoute l'arobase au dictionnaire
                arobases[arobase] = {
                    "occurence": 1,
                    "id": [tweet.id]
                }
    
    # On retourne les arobases avec leur nombre de publications
    return arobases

def top_arobases(k:int, arobases: dict) -> list:
    """
    Cette fonction permet de retourner les k arobase les plus utilisés.
    :param k: Nombre d'arobases à retourner
    :param arobases: Liste des arobases avec leur nombre de publications
    :return top_arobases: Liste des k arobase les plus utilisés
    """
    
    # On retourne les k arobase les plus utilisés
    return sorted(arobases, key=lambda arobase: arobases[arobase]["occurence"], reverse=True)[:k]

def tweetid_arobases(user: str, arobases: dict) -> list:
    """
    Cette fonction permet de retourner les id des tweets qui ont utilisé un arobas donné.
    :param user: Arobase à rechercher
    :param arobases: Liste des arobases avec leur nombre de publications
    :return tweetid_arobases: Liste des id des tweets qui ont utilisé l'arobase
    """
    
    # On retourne les id des tweets qui ont utilisé l'arobase
    return arobases[user]["id"]

In [4]:
def main(filename: str):
    """
    Fonction principale
    :param filename: Nom du fichier json  
    """
    
    tweets = [] # Liste des tweets
    
    # On ouvre le fichier json et on le charge dans une liste de dictionnaire
    with open(filename, "r", encoding="UTF-8") as file:
            data = [json.loads(line) for line in file]

    # On parcourt chaque tweet
    for tweet_data in data:
        tweet = Tweet(tweet_data) # On crée un objet Tweet
        tweet.extract_entities() # On extrait les entités du tweet (hashtags, arobase, sentiment)
        tweets.append(tweet) # On ajoute le tweet à la liste des tweets
    
    # On affiche les hashtags avec leur nombre d'occurence
    print("Les hashtags avec leur nombre de publications sont :")
    for hashtag, occurence in count_hashtags(tweets).items():
        print(f"{hashtag} : {occurence}")
        
    # On affiche les 10 hashtags les plus utilisés
    print("\nLes 10 hashtags les plus utilisés sont :")
    for hashtag in top_hashtags(10, count_hashtags(tweets)):
        print(hashtag)
        
    # On affiche les arobases avec leur nombre d'occurence
    print("\nLes arobase avec leur nombre de publications sont :")
    for arobase, occurence in count_arobases(tweets).items():
        print(f"{arobase} : {occurence}")
        
    # On affiche les 10 arobases les plus utilisés
    print("\nLes 10 arobase les plus utilisés sont :")
    for arobase in top_arobases(10, count_arobases(tweets)):
        print(arobase)
        
    # On affiche les id des tweets qui ont utilisé l'arobase @IainLJBrown
    print("\nLes id des tweets qui ont utilisé l'arobase @IainLJBrown sont :")
    for tweetid in tweetid_arobases("IainLJBrown", count_arobases(tweets)):
        print(tweetid)
        
    print(tweets[54].cleaned_text)
            
if __name__ == "__main__":
    main("aitweets.json")

Les hashtags avec leur nombre de publications sont :
hdatasystems : 1
Artificia : 7
MachineLearning : 84
DataScience : 50
Python : 17
AI : 200
100DaysOfCode : 15
DEVCommunity : 2
IoT : 15
flutte : 1
artificialintelligence : 43
ArtificialIntelligence : 195
DeepLearning : 20
FEATURED : 19
COURSES : 19
machine : 12
learning : 24
data : 10
science, : 2
Tensorflow, : 2
artificial : 8
intelligence, : 2
python : 5
100daysofcode : 2
coding : 12
udemy : 3
Artificial_Intelligence : 10
ai : 52
ml : 9
dl : 8
Intela : 1
Ar : 2
ArtificialIntelligenc : 1
GovCloudWorldNews : 1
Java : 7
Bitcoin : 1
SciketLearn : 1
ComputerVision : 3
DeepLearningFramework : 1
AutonomousVehicles : 3
NFT : 4
ImageProcessing : 2
React : 2
LinearAlgebra : 2
Bluetooth : 1
Robotics : 6
Automation : 7
Ro : 2
REALITY : 1
BigData : 25
Analytics : 16
DataScien : 4
URL : 1
python3 : 13
datatype : 11
programming : 16
javascript : 6
Intelligence : 8
Unity : 8
Itamaraty : 1
cybersec : 1
privacy : 2
bigdata : 7
behavioraleconomics : 1